# ĐỀ TÀI: Ứng dụng Phân tích Cơ bản, Kỹ thuật và Machine Learning trong Xây dựng Chiến lược Giao dịch Cổ phiếu

#### **Giới thiệu**

Trong bài làm này, nhóm tiến hành xây dựng một **chiến lược lựa chọn và giao dịch cổ phiếu** dựa trên dữ liệu tài chính và kỹ thuật của thị trường chứng khoán Việt Nam.  
Ý tưởng cốt lõi là kết hợp **phân tích cơ bản (FA)**, **phân tích kỹ thuật (TA)** và **máy học (ML)** để tạo ra một hệ thống đánh giá toàn diện cho từng mã cổ phiếu.  

Quy trình được thiết kế theo các bước:  
1. **Thu thập & tiền xử lý dữ liệu**: giá cổ phiếu daily, báo cáo tài chính, chỉ số ngành.  
2. **Xây dựng các chỉ số FA & TA** → gom thành **FA_Score** và **TA_Score** chuẩn hóa về thang 0–100.  
3. **Gán nhãn dữ liệu & huấn luyện mô hình ML (Random Forest)** để dự báo xác suất tăng giá trong tương lai.  
4. **Tạo ML_Score** cho toàn bộ dữ liệu và sử dụng trong **chiến lược giao dịch Long–Short**.  
5. **Backtest & đánh giá hiệu suất** → đo lường bằng CAGR, Sharpe Ratio, Max Drawdown.  

Kết quả cuối cùng cho thấy mô hình có khả năng tạo lợi nhuận vượt trội và kiểm soát rủi ro tốt, thể hiện tiềm năng ứng dụng thực tế trong đầu tư.  


In [43]:
# # Cài đặt các thư viện cần thiết

# !pip install pandas numpy scipy matplotlib seaborn dataclasses
# !pip install scikit-learn
# !pip install yfinance requests
# !pip install openpyxl xlrd
# !pip install --extra-index-url https://fiinquant.github.io/fiinquantx/simple fiinquantx

In [ ]:
# # ĐĂNG NHẬP FIINQUANT

# from FiinQuantX import FiinSession

# username = 'username'
# password = 'password'

# client = FiinSession(
#     username=username,
#     password=password,
# ).login()

## **1. Thu Thập Dữ Liệu từ FiinQuant**

**Nguồn dữ liệu:** Ba sàn giao dịch chính của TTCK Việt Nam
  - **HOSE** (Ho Chi Minh Stock Exchange) - VNINDEX
  - **HNX** (Hanoi Stock Exchange) - HNXINDEX  
  - **UPCOM** (Unlisted Public Company Market) - UPCOMINDEX.

**Khoảng thời gian:** 01/01/2023 đến 31/08/2025  
**Tần suất:** Dữ liệu ngày (daily frequency)  
**Loại dữ liệu:** Dữ liệu lịch sử

**Các trường dữ liệu chính:**
- **OHLCV**: Open, High, Low, Close, Volume (dữ liệu giá và khối lượng cơ bản)
- **Trading metrics**: Khối lượng giao dịch, giá trị giao dịch
- **Foreign flows**: Chỉ số mua/bán ròng của nhà đầu tư nước ngoài (NN)
- **Market identifiers**: Mã cổ phiếu, sàn giao dịch, timestamp

**File output:** **`all_stocks.csv`** 

```python
import datetime

tickers1 = list(client.TickerList(ticker="VNINDEX"))
tickers2 = list(client.TickerList(ticker="HNXINDEX"))
tickers3 = list(client.TickerList(ticker="UPCOMINDEX"))
tickers = tickers1 + tickers2 + tickers3

df = client.Fetch_Trading_Data(
    realtime=False,
    tickers=tickers,
    fields=['open','high','low','close','volume','bu','sd','fb','fs','fn'],
    adjusted=True,
    by="1d",
    from_date="2023-01-01",
    to_date=datetime.datetime.now()
).get_data()

#### **Lọc tickers hợp lệ và lấy dữ liệu Fundamental Analysis (FA)**

Do ETF và chứng chỉ quỹ không có báo cáo tài chính như công ty niêm yết, nên các chỉ số phân tích cơ bản (FA) như ROE, P/E, EPS Growth… không áp dụng. Ta lọc những mã có thể lấy báo cáo tài chính để crawl FA.

Khi lấy dữ liệu từ FiinQuant API, một số tickers có thể không có đầy đủ dữ liệu FA hoặc gặp lỗi khi crawl. Do đó:
- **good**: Danh sách các tickers lấy được đầy đủ dữ liệu FA thành công
- **bad**: Danh sách các tickers gặp lỗi khi crawl hoặc không có dữ liệu FA

**File output:** **`all_fa.csv`**


```python
# lọc các mã không có FA (có thể xác định quy tắc)
tickers_stocks = [t for t in tickers if not t.startswith("FU") and not t.endswith("C")]

# Lấy dữ liệu FA
import pandas as pd

good, bad = [], []

for t in list(tickers_stocks):
    try:
        tmp = client.FundamentalAnalysis().get_ratios(
            tickers=[t],
            TimeFilter="Yearly",     
            LatestYear=2025,
            NumberOfPeriod=3,       
            Consolidated=True,
            Fields=None
        )
        if tmp:
            good.extend(tmp)
    except:
        bad.append(t)

df_ratios = pd.DataFrame(good)

## **2. Tiền Xử Lý Dữ Liệu**

Do các chỉ số tài chính được lưu dưới dạng JSON trong cột `ratios`, cần:
- Parse JSON string thành dictionary
- Flatten nested structure để có các cột riêng biệt

**File output**: **`FA_DATA.csv`**

```python
import ast
df["ratios"] = df["ratios"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
expanded = pd.json_normalize(df["ratios"])
df = pd.concat([df.drop(columns=["ratios"]), expanded], axis=1)

# chỉ lấy năm 2023, 2024, 2025
df = df[df["year"].isin([2023, 2024, 2025])]
df["isTTM"] = df["isTTM"].fillna(False).astype(int)

Trong bài toán này, dữ liệu báo cáo tài chính (FA) không thể sử dụng ngay khi kỳ kế toán kết thúc, mà chỉ được dùng sau khi doanh nghiệp công bố ra thị trường. Nếu không xử lý, khi backtest sẽ vô tình dùng dữ liệu chưa được công bố → dẫn đến data leakage (nhìn trước tương lai), khiến kết quả backtest ảo tưởng và không thực tế.

Do đó, cần **xây dựng một quy tắc ngày công bố giả định theo deadline** chuẩn của HOSE/HNX/UPCOM:
- Q1: công bố chậm nhất 30/04 (sau 45 ngày).
- Q2: công bố chậm nhất 31/07 (sau 45 ngày).
- Q3: công bố chậm nhất 31/10 (sau 45 ngày).
- Q4 (BCTC năm): công bố chậm nhất 31/03 năm sau (sau 90 ngày).

```python
def create_assumed_report_date(df):
    """
    Hàm tính ngày công bố giả định dựa trên quý và năm.
    """
    def get_report_date(row):
        year = row["year"]
        quarter = row["quarter"]

        if quarter == 1:
            return pd.Timestamp(f"{year}-04-30")
        elif quarter == 2:
            return pd.Timestamp(f"{year}-07-31")
        elif quarter == 3:
            return pd.Timestamp(f"{year}-10-31")
        elif quarter == 4:
            return pd.Timestamp(f"{year + 1}-03-31")
        else:
            return pd.NaT  

## **3. Xây Dựng FA Score (Quality-Growth-Value)**

Tính `FA_Score` cho từng cổ phiếu, dựa trên 3 nhóm:  
- **Q (Quality):** ROE, ROA  
- **G (Growth):** EPS, Doanh thu YoY  
- **V (Value):** P/E, P/B  

Công thức: 
$$FA\_Score = 0.4 \times Q + 0.3 \times G + 0.3 \times V$$

`FA_Score` cho phép lọc ra nhóm Top % cổ phiếu tốt và hợp lý về cả chất lượng doanh nghiệp lẫn mức định giá. Đây sẽ là tập "universe" ban đầu trước khi áp dụng các tín hiệu phân tích kỹ thuật (TA) để xác định thời điểm mua bán tối ưu.


```python
from sklearn.preprocessing import MinMaxScaler

def compute_fa_score_safe(df):
    df = df.copy()

    df["Q"] = df[["ProfitabilityRatio.ROE", "ProfitabilityRatio.ROA"]].mean(axis=1, skipna=True)
    df["G"] = df[["ValuationRatios.BasicEPS", "Growth.NetRevenueGrowthYoY"]].mean(axis=1, skipna=True)

    inv_PE = 1 / df["ValuationRatios.PriceToEarning"].replace(0, 1e-6)
    inv_PB = 1 / df["ValuationRatios.PriceToBook"].replace(0, 1e-6)
    df["V"] = pd.concat([inv_PE, inv_PB], axis=1).mean(axis=1, skipna=True)

    # Chuẩn hóa theo từng năm để tránh data leakage
    df[["Q", "G", "V"]] = df.groupby("year")[["Q", "G", "V"]].transform(
        lambda x: MinMaxScaler().fit_transform(x.fillna(0).values.reshape(-1, 1)).flatten()
    )

    # Tính FA_Score (yêu cầu ít nhất 2/3 chỉ số hợp lệ)
    weights = {"Q": 0.4, "G": 0.3, "V": 0.3}
    df["FA_Score"] = df[["Q", "G", "V"]].apply(
        lambda row: (
            sum(row[col] * weights[col] for col in ["Q", "G", "V"] if pd.notnull(row[col]))
            / sum(weights[col] for col in ["Q", "G", "V"] if pd.notnull(row[col]))
        ) if row.notnull().sum() >= 2 else None,
        axis=1
    )

    return df.sort_values(["year", "FA_Score"], ascending=[True, False])[["ticker", "year", "quarter", "FA_Score"]]

#### **Bổ sung thông tin `Sector` và lọc cổ phiếu theo `FA_Score`**

- Nhóm đã chủ động **thu thập và chuẩn hóa dữ liệu ngành (sector)** cho từng mã cổ phiếu, lưu trong file `ticker_category.xlsx`.  
- Thông tin sector được **map vào DataFrame** để phục vụ phân tích nhóm ngành và phân bổ danh mục hợp lý.  
- Sau đó, trong mỗi sector, nhóm **lọc ra nhóm cổ phiếu thuộc top 40% có FA_Score cao nhất**.  

Kết quả:
- Giúp so sánh cổ phiếu **trong cùng ngành**, thay vì toàn thị trường (tránh thiên lệch do đặc thù ngành).  
- Đảm bảo chiến lược tập trung vào **các doanh nghiệp cơ bản tốt nhất của từng sector**, tạo universe chất lượng trước khi áp dụng TA và ML.  


```python
import math

def filter_top_40pct_by_sector(df):
    top_stocks = []

    for sector, group in df.groupby("category"):
        group_sorted = group.sort_values(by="FA_Score", ascending=False)
        top_n = math.ceil(len(group_sorted) * 0.4)
        top_group = group_sorted.head(top_n)
        top_stocks.append(top_group)

    return pd.concat(top_stocks).reset_index(drop=True)

top_40pct_by_category = filter_top_40pct_by_sector(top_fa_merged)

print(top_40pct_by_category[["ticker", "category", "FA_Score"]])

#### **Kết hợp dữ liệu `FA_Score` với dữ liệu giá daily**

- Nhóm thực hiện **merge DataFrame FA_Score với DataFrame giá daily (`all_stocks.csv`)** để có được bộ dữ liệu hoàn chỉnh cả về yếu tố cơ bản (FA) và thị trường (giá, volume).  
- Sau khi nối dữ liệu, một số dòng sẽ **không có FA_Score** do báo cáo tài chính chưa được công bố (ví dụ dữ liệu giá năm 2024–2025 nhưng báo cáo chưa ra).

Giải pháp xử lý: **Forward Fill (ffill)**  
- Khi một năm chưa có báo cáo, nhà đầu tư thực tế sẽ **dựa trên FA_Score của năm gần nhất đã công bố**.  
- Do đó, nhóm áp dụng phương pháp Forward Fill để gán FA_Score gần nhất cho các dòng thiếu.  

**File output:** **`final_fa.csv`**

``` python  
# Kiểm tra phân bố dữ liệu thiếu theo năm
missing_fa = final_dataset[final_dataset['FA_Score'].isna()]
print("Phân bố dữ liệu thiếu FA_Score theo năm:")
print(missing_fa['year'].value_counts().sort_index())
print()

# Forward Fill 
final_dataset_ff = final_dataset.copy()
final_dataset_ff['FA_Score_filled'] = final_dataset_ff.groupby('ticker')['FA_Score'].fillna(method='ffill')

## **4. Xây Dựng Technical Analysis (TA) Score (Trend + Flow)**

Sử dụng các hàm có sẵn của FiinQuant, tự xây dựng những hàm chưa có để tính toán các chỉ số VMA, SMA, OBV, ATR, MACD

``` python   
def vma(df, window=20):
    df = df.copy()
    df["VMA" + str(window)] = df.groupby("ticker")["volume"].transform(lambda x: x.rolling(window).mean())
    return df

``` python  

fi = client.FiinIndicator()
df['SMA20'] = fi.sma(df['close'], window = 20)
df['SMA50'] = fi.sma(df['close'], window = 50)
df = vma(df, window = 20)
df['OBV'] = fi.obv(df['close'], df['volume'])
df['ATR14'] = fi.atr(df['high'], df['low'], df['close'], window=14)
df["High20"] = df["close"].rolling(20).max()
df["Peak"] = df["close"].shift(1).rolling(window=20).max()

``` python  
df['MACD'] = fi.macd(df['close'], window_fast=12, window_slow=26)
df['Signal'] = fi.macd_signal(df['close'], window_fast=12, window_slow=26, window_sign=9)

#### **Tính `TA_Score` (0–100):**
Ta kết hợp các yếu tố:
- `Trend_Score` (0–50): dựa trên SMA20 > SMA50, MACD > Signal, và giá > SMA50 → đo sức mạnh xu hướng.
- `Flow_Score` (0–50): dựa trên khối lượng > 1.5×VMA20, OBV tăng, và mua ròng nước ngoài → đo dòng tiền.

Công thức:
$$TA\_Score = Trend\_Score + Flow\_Score$$

```python  
df["Trend_Score"] = (
    (df["SMA20"] > df["SMA50"]).astype(int) +
    (df["MACD"] > df["Signal"]).astype(int) +
    (df["close"] > df["SMA50"]).astype(int)
) / 3 * 50   

df["Flow_Score"] = (
    (df["volume"] > 1.5 * df["VMA20"]).astype(int) +
    (df["OBV"].diff(5) > 0).astype(int) +
    (df["fn"] > 0).astype(int)
) / 3 * 50

df["TA_Score"] = df["Trend_Score"] + df["Flow_Score"]   # 0–100

Tạo **điều kiện mua vào** khi cổ phiếu thoả 3 tiêu chí:
- `TA_Score` ≥ 60 → xu hướng + dòng tiền đủ mạnh.
- `close` > `High20` → breakout khỏi đỉnh 20 phiên.
- `ATR%` ≤ 6% → biến động giá không quá cao, rủi ro vừa phải.

```python  
df["Entry"] = (
    (df["TA_Score"] >= 60) &
    (df["close"] > df["High20"]) &
    ((df["ATR14"] / df["close"]) <= 0.06)
)

**Điều kiện bán ra / thoát vị thế**:
- `close` < `SMA20` → giá gãy SMA20, tín hiệu suy yếu.
- Trailing stop `2×ATR` từ đỉnh gần nhất → bảo toàn lợi nhuận, hạn chế thua lỗ lớn.

```python
df["Exit"] = (df["close"] < df["SMA20"]) | (df["close"] < df["Peak"] - 2 * df["ATR14"])

**File output:** **`final_ta.csv`**

#### **Tạo bộ dữ liệu tổng hợp FA + TA**

Sau khi chuẩn hoá dữ liệu và xử lý thiếu FA_Score bằng phương pháp Forward Fill, nhóm tiến hành **ghép dữ liệu FA (Fundamental Analysis) với dữ liệu TA (Technical Analysis)**.  

Kết quả là một DataFrame hoàn chỉnh bao gồm:  
  - Các chỉ số tài chính (FA_Score).  
  - Các chỉ báo kỹ thuật (TA_Score, SMA, MACD, OBV, ATR,...).  
  - Thông tin giá và khối lượng giao dịch.  

**Bộ dữ liệu cuối cùng** này được lưu lại dưới tên **`fa_ta.csv`**, đóng vai trò làm **nguồn dữ liệu đầu vào duy nhất** cho các bước tiếp theo:  
- Lựa chọn cổ phiếu theo tiêu chí FA + TA.  
- Chạy backtest chiến lược.  
- Huấn luyện mô hình Machine Learning nâng cao.  


## **5. Regime Filter & FA-TA Combination**

In [45]:
import pandas as pd
import numpy as np
from dataclasses import dataclass

In [46]:
df = pd.read_csv("fa_ta.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,ticker,timestamp,open,high,low,close,volume,bu,...,MACD,Signal,Trend_Score,Flow_Score,TA_Score,ATR14,High20,Peak,Entry,Exit
0,0,0,AAA,2023-01-03,6539.643,6866.145,6539.643,6866.145,1543984.0,938600.0,...,NaN,NaN,0.0,16.666667,16.666667,NaN,NaN,NaN,False,False
1,1,1,AAA,2023-01-04,6866.145,7000.587,6827.733,6827.733,1302505.0,462900.0,...,NaN,NaN,0.0,16.666667,16.666667,NaN,NaN,NaN,False,False
2,2,2,AAA,2023-01-05,6866.145,6904.557,6808.527,6885.351,980473.0,487200.0,...,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN,False,False
3,3,3,AAA,2023-01-06,6885.351,6990.984,6818.130,6856.542,1431699.0,564300.0,...,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN,False,False
4,4,4,AAA,2023-01-09,6914.160,6962.175,6760.512,6789.321,1121385.0,414000.0,...,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN,False,False


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855053 entries, 0 to 855052
Data columns (total 32 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0.1  855053 non-null  int64  
 1   Unnamed: 0    855053 non-null  int64  
 2   ticker        855053 non-null  object 
 3   timestamp     855053 non-null  object 
 4   open          855053 non-null  float64
 5   high          855053 non-null  float64
 6   low           855053 non-null  float64
 7   close         855053 non-null  float64
 8   volume        855053 non-null  float64
 9   bu            850422 non-null  float64
 10  sd            850422 non-null  float64
 11  fb            855053 non-null  float64
 12  fs            855053 non-null  float64
 13  fn            855053 non-null  float64
 14  year          855053 non-null  int64  
 15  category      855053 non-null  object 
 16  quarter       339553 non-null  float64
 17  FA_Score      855053 non-null  float64
 18  SMA2

In [48]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

#### **Regime Filter: Xác định chế độ thị trường**

1. `make_fallback_index_from_prices(df)`
- Tạo chỉ số giả lập (equal-weighted) bằng cách lấy trung bình **open, high, low, close** theo ngày.  
- Dùng khi không có dữ liệu VNINDEX để tham chiếu xu hướng chung.  

2. `compute_adx(high, low, close, period=14)`
- Tính toán **ADX (Average Directional Index)** theo công thức Wilder.  
- Các bước: tính TR, +DM, –DM → Wilder smoothing → +DI, –DI → DX → ADX.  
- Ý nghĩa: ADX đo **độ mạnh xu hướng** (ADX > 20 = có trend).  

3. `compute_regime(index_df, adx_thresh=20)`
- Phân loại thị trường thành 3 chế độ:  
  - **Bull**: close > MA200 và ADX > 20  
  - **Bear**: close < MA200 và slope(MA200) < 0  
  - **Sideways**: còn lại  
- Output: DataFrame gồm `timestamp, regime, MA200, ADX14`.  

In [49]:
def make_fallback_index_from_prices(df):
    """Tạo index giả: equal-weight trung bình close theo ngày."""
    idx = (df
           .groupby('timestamp', as_index=False)
           .agg(open=('open','mean'),
                high=('high','mean'),
                low=('low','mean'),
                close=('close','mean')))
    idx = idx.sort_values('timestamp').reset_index(drop=True)
    return idx

def compute_adx(high, low, close, period=14):
    """Trả về ADX theo công thức chuẩn (Wilder)"""
    high = high.values; low = low.values; close = close.values
    n = len(close)
    tr = np.zeros(n); plus_dm = np.zeros(n); minus_dm = np.zeros(n)
    for i in range(1, n):
        tr[i] = max(high[i] - low[i], abs(high[i] - close[i - 1]), abs(low[i] - close[i - 1]))
        up_move = high[i] - high[i - 1]
        down_move = low[i - 1] - low[i]
        plus_dm[i] = up_move if (up_move > down_move and up_move > 0) else 0
        minus_dm[i] = down_move if (down_move > up_move and down_move > 0) else 0

    # Wilder smoothing
    def wilder_smooth(arr, p):
        out = np.zeros_like(arr)
        out[:p] = np.nan
        out[p] = np.nansum(arr[1:p+1])
        for i in range(p + 1, n):
            out[i] = out[i - 1] - (out[i - 1]/p) + arr[i]
        return out

    tr14 = wilder_smooth(tr, period)
    plus_dm14 = wilder_smooth(plus_dm, period)
    minus_dm14 = wilder_smooth(minus_dm, period)

    plus_di = 100 * (plus_dm14 / tr14)
    minus_di = 100 * (minus_dm14 / tr14)
    dx = 100 * np.abs((plus_di - minus_di) / (plus_di + minus_di))
    # ADX = Wilder smoothing của DX
    adx = np.zeros_like(dx); adx[:period*2] = np.nan
    # seed
    seed = np.nanmean(dx[period+1:period*2+1])
    adx[period*2] = seed
    for i in range(period*2+1, n):
        adx[i] = (adx[i-1] * (period-1) + dx[i]) / period
    return pd.Series(adx)

def compute_regime(index_df, adx_thresh=20):
    """Gán regime: Bull / Sideways / Bear từ VNINDEX."""
    idx = index_df.sort_values('timestamp').reset_index(drop=True).copy()
    idx['MA200'] = idx['close'].rolling(200, min_periods=200).mean()
    idx['ADX14'] = compute_adx(idx['high'], idx['low'], idx['close'], period=14)
    # slope MA200 (5 ngày)
    idx['MA200_slope'] = idx['MA200'].diff(5)

    cond_bull = (idx['close'] > idx['MA200']) & (idx['ADX14'] > adx_thresh)
    cond_bear = (idx['close'] < idx['MA200']) & (idx['MA200_slope'] < 0)

    idx['regime'] = np.select(
        [cond_bull, cond_bear],
        ['Bull', 'Bear'],
        default='Sideways'
    )
    return idx[['timestamp','regime','MA200','ADX14']]

#### **Gắn nhãn `regime` (Bull / Sideways / Bear) vào từng dòng dữ liệu cổ phiếu.**

Các bước:
1. Chuẩn hóa và sắp xếp `timestamp, ticker`.  
2. Nếu **chưa có index_df** (VNINDEX), tạo fallback index bằng giá trung bình equal-weight (`make_fallback_index_from_prices`).  
3. Tính toán **regime** từ index_df bằng `compute_regime`.  
4. Merge kết quả vào dữ liệu cổ phiếu theo `timestamp`.  
5. Với các ngày thiếu regime → mặc định là **Sideways**.  

Ý nghĩa:
- Giúp mỗi dòng dữ liệu giá cổ phiếu biết thị trường chung đang ở chế độ nào.  
- Là bước chuẩn bị cần thiết để tính **Final_Score = f(FA, TA, Regime)**.  
- Đảm bảo chiến lược điều chỉnh trọng số hợp lý theo bối cảnh thị trường.


In [50]:
def attach_regime_to_df(df, index_df=None):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['timestamp','ticker']).reset_index(drop=True)

    if index_df is None:
        index_df = make_fallback_index_from_prices(df)
    index_df = index_df.copy()
    index_df['timestamp'] = pd.to_datetime(index_df['timestamp'])

    regime_df = compute_regime(index_df)
    out = df.merge(regime_df, on='timestamp', how='left')
    # Nếu vẫn thiếu, mặc định Sideways
    out['regime'] = out['regime'].fillna('Sideways')
    return out

#### **Kết hợp `FA_Score` và `TA_Score` thành `Final_Score`, có điều chỉnh theo **regime thị trường**.**

1. **Chuẩn hóa FA và TA về thang điểm 0–100** bằng `rescale_0_100`.  
2. **Trọng số theo chế độ thị trường (regime):**
   - **Bull:** ưu tiên TA (0.3·FA + 0.7·TA).  
   - **Bear:** ưu tiên FA (0.7·FA + 0.3·TA).  
   - **Sideways:** cân bằng (0.5·FA + 0.5·TA).  

3. **Tính Final_Score:**  
$$Final\_Score = w_{FA} \times FA + w_{TA} \times TA$$

-> Final_Score là thước đo tổng hợp cuối cùng để xếp hạng và chọn cổ phiếu.

In [51]:
def rescale_0_100(s):
    s = s.astype(float)
    if s.max() == s.min():
        return pd.Series(50.0, index=s.index)
    return 100 * (s - s.min()) / (s.max() - s.min())

def add_final_score(df):
    df = df.copy()
    # Đảm bảo thang 0-100
    fa = rescale_0_100(df['FA_Score'])
    ta = rescale_0_100(df['TA_Score'])

    wFA = np.where(df['regime']=='Bull', 0.3,
          np.where(df['regime']=='Bear', 0.7, 0.5))
    wTA = 1 - wFA

    df['Final_Score'] = wFA*fa + wTA*ta
    return df

#### **Backtest Chiến Lược**

Hàm `backtest(df, cfg)` mô phỏng quá trình giao dịch dựa trên các tín hiệu **Entry/Exit** đã tính toán.  

1. **Thiết lập cấu hình (`BTConfig`)**  
   - `top_n`: số lượng cổ phiếu tối đa được giữ.  
   - `max_weight`: giới hạn tỷ trọng mỗi mã.  
   - `fee`, `slippage`: chi phí giao dịch.  
   - `init_capital`: vốn khởi đầu.  

2. **Vòng lặp backtest theo ngày**  
   - **Bán**: thoát vị thế nếu có tín hiệu Exit, SMA20 gãy hoặc trailing stop (giá < peak − 2×ATR).  
   - **Mua**: chọn ứng viên có tín hiệu Entry, ưu tiên Final_Score cao, phân bổ vốn theo tỷ trọng.  
   - **Cập nhật NAV**: tính giá trị tài sản ròng (NAV) mỗi ngày, lưu lại lịch sử giao dịch.  

3. **Kết quả trả về**  
   - `nav_df`: chuỗi thời gian NAV, vốn tiền mặt và số lượng vị thế.  
   - `trades_df`: danh sách lệnh mua/bán đã thực hiện (timestamp, ticker, side, giá, khối lượng).  

In [52]:
@dataclass
class BTConfig:
    top_n: int = 15
    max_weight: float = 0.15   
    fee: float = 0.002         
    slippage: float = 0.0005  
    init_capital: float = 1_000_000_000 # 1 tỷ
    use_existing_entry_exit: bool = True

def backtest(df, cfg: BTConfig):
    data = df.copy()
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.sort_values(['timestamp','ticker']).reset_index(drop=True)

    # Regime -> Final_Score -> Entry/Exit
    data = attach_regime_to_df(data)                     
    data = add_final_score(data)

    dates = data['timestamp'].drop_duplicates().sort_values().tolist()
    holdings = {}  
    cash = cfg.init_capital
    nav_records = []
    trades = []

    # Tiện ích
    def portfolio_value_at(d):
        px = day_df.set_index('ticker')['close']
        val = cash
        for t, pos in holdings.items():
            if t in px.index:
                val += pos['shares'] * px.loc[t]
        return val

    for d in dates:
        day_df = data[data['timestamp']==d]
        day_prices = day_df.set_index('ticker')

        # 1) BÁN: exit signal hoặc trailing stop 2*ATR từ peak
        to_sell = []
        for t, pos in holdings.items():
            if t not in day_prices.index:  # không có giá -> bỏ qua
                continue
            c = float(day_prices.loc[t, 'close'])
            atr = float(day_prices.loc[t, 'ATR14']) if 'ATR14' in day_prices.columns else np.nan
            sma20 = float(day_prices.loc[t, 'SMA20']) if 'SMA20' in day_prices.columns else np.nan
            # update peak
            pos['peak'] = max(pos['peak'], c)

            # điều kiện exit
            rule_exit_flag = 0
            if 'Exit' in day_prices.columns:
                rule_exit_flag = int(day_prices.loc[t, 'Exit'])

            trailing_stop_hit = (not np.isnan(atr)) and (c < pos['peak'] - 2*atr)
            sma_break = (not np.isnan(sma20)) and (c < sma20)

            if rule_exit_flag or trailing_stop_hit or sma_break:
                to_sell.append(t)

        # Thực thi bán trước (giải phóng tiền)
        for t in to_sell:
            c = float(day_prices.loc[t, 'close'])
            qty = holdings[t]['shares']
            sell_px = c * (1 - cfg.fee - cfg.slippage)
            proceeds = qty * sell_px
            cash += proceeds
            trades.append({'timestamp': d, 'ticker': t, 'side': 'SELL', 'price': sell_px, 'qty': qty})
            del holdings[t]

        # 2) MUA: chọn ứng viên Entry hôm nay, rank theo Final_Score, lấy tối đa top_n 
        slots_left = cfg.top_n - len(holdings)
        if slots_left > 0:
            candidates = day_df[(day_df['Entry'] == 1) & (~day_df['ticker'].isin(holdings.keys()))].copy()
            if not candidates.empty:
                candidates = candidates.sort_values('Final_Score', ascending=False).head(slots_left)

                # Phân bổ trọng số theo Final_Score, có cap max_weight
                scores = candidates['Final_Score'].clip(lower=0)
                if scores.sum() == 0:
                    weights = np.repeat(1.0/len(candidates), len(candidates))
                else:
                    weights = scores / scores.sum()

                weights = np.minimum(weights, cfg.max_weight)
                weights = weights / weights.sum()  # renormalize

                # Tổng tài sản hiện tại (sau khi bán)
                port_val = portfolio_value_at(d)

                for (idx, row), w in zip(candidates.iterrows(), weights):
                    t = row['ticker']; c = float(row['close'])
                    buy_budget = port_val * w
                    buy_px = c * (1 + cfg.fee + cfg.slippage)
                    qty = int(buy_budget // buy_px)
                    if qty <= 0: 
                        continue
                    cost = qty * buy_px
                    if cost > cash:
                        # nếu thiếu tiền, giảm qty
                        qty = int(cash // buy_px)
                        if qty <= 0:
                            continue
                        cost = qty * buy_px

                    cash -= cost
                    holdings[t] = {
                        'shares': qty,
                        'entry_price': buy_px,
                        'peak': c
                    }
                    trades.append({'timestamp': d, 'ticker': t, 'side': 'BUY', 'price': buy_px, 'qty': qty})

        # 3) Ghi NAV cuối ngày
        nav = portfolio_value_at(d)
        nav_records.append({'timestamp': d, 'NAV': nav, 'cash': cash, 'n_positions': len(holdings)})

    nav_df = pd.DataFrame(nav_records).sort_values('timestamp')
    
    # xử lý trường hợp không có giao dịch
    if len(trades) == 0:
        trades_df = pd.DataFrame(columns=['timestamp', 'ticker', 'side', 'price', 'qty'])
    else:
        trades_df = pd.DataFrame(trades).sort_values('timestamp')

    return nav_df, trades_df

#### **Đánh Giá Hiệu Suất Chiến Lược**

Hàm `performance_metrics(nav_df, rf=0.0)` tính các chỉ số tài chính quan trọng từ kết quả backtest.  

##### Các chỉ số tính toán:
- **CAGR (Compound Annual Growth Rate)**  
  Tốc độ tăng trưởng kép hàng năm của danh mục.  

- **Max Drawdown**  
  Mức sụt giảm lớn nhất so với đỉnh trước đó → đo lường rủi ro.  

- **Annualized Return (AnnReturn)**  
  Lợi nhuận trung bình năm (quy đổi từ lợi nhuận ngày).  

- **Annualized Volatility (AnnVol)**  
  Độ biến động (rủi ro) hằng năm, chuẩn hóa từ dữ liệu ngày.  

- **Sharpe Ratio**  
  Tỷ số lợi nhuận/rủi ro, so sánh với lãi suất phi rủi ro (`rf`).  


In [53]:
def performance_metrics(nav_df, rf=0.0):
    nav = nav_df.set_index('timestamp')['NAV'].astype(float)
    ret = nav.pct_change().fillna(0.0)

    # CAGR
    n_days = (nav.index.max() - nav.index.min()).days
    years = max(n_days/365.25, 1e-9)
    cagr = (nav.iloc[-1] / nav.iloc[0])**(1/years) - 1 if len(nav)>1 else 0.0

    # Max Drawdown
    roll_max = nav.cummax()
    dd = nav/roll_max - 1
    maxdd = dd.min()

    # Sharpe (252 phiên)
    ann_ret = (1+ret.mean())**252 - 1
    ann_vol = ret.std(ddof=0) * np.sqrt(252)
    sharpe = (ann_ret - rf) / (ann_vol + 1e-9)

    return {
        'CAGR': cagr,
        'MaxDD': maxdd,
        'AnnReturn': ann_ret,
        'AnnVol': ann_vol,
        'Sharpe': sharpe
    }

#### **Chạy thử backtest**

In [54]:
cfg = BTConfig(top_n=15, max_weight=0.15, fee=0.002, slippage=0.0005, init_capital=1_000_000_000)

nav_df, trades_df = backtest(df, cfg)
metrics = performance_metrics(nav_df)

print("📊 KẾT QUẢ BACKTEST:")
print(f"CAGR: {metrics['CAGR']:.2%}")
print(f"Max Drawdown: {metrics['MaxDD']:.2%}")
print(f"Sharpe Ratio: {metrics['Sharpe']:.2f}")
print(f"Annual Return: {metrics['AnnReturn']:.2%}")
print(f"Annual Volatility: {metrics['AnnVol']:.2%}")
print(f"Số lệnh: {len(trades_df)}")
print()

print("📈 NAV cuối cùng:")
display(nav_df.tail())

if len(trades_df) > 0:
    print("💰 Giao dịch đầu tiên:")
    display(trades_df.head(10))
else:
    print("Không có giao dịch nào được thực hiện!")

📊 KẾT QUẢ BACKTEST:
CAGR: 0.00%
Max Drawdown: 0.00%
Sharpe Ratio: 0.00
Annual Return: 0.00%
Annual Volatility: 0.00%
Số lệnh: 0

📈 NAV cuối cùng:


,timestamp,NAV,cash,n_positions
657,2025-08-25,1000000000,1000000000,0
658,2025-08-26,1000000000,1000000000,0
659,2025-08-27,1000000000,1000000000,0
660,2025-08-28,1000000000,1000000000,0
661,2025-08-29,1000000000,1000000000,0


Không có giao dịch nào được thực hiện!


Sau khi chạy backtest, nhận thấy điều kiện Entry là close > High20 = 0 dòng ->  khó vượt qua -> **Thay đổi điều kiện Entry** linh hoạt hơn.
- **Điều kiện 1:**  TA_Score >= 50 (thay vì 60)
- **Điều kiện 2:** Gần đỉnh 20 ngày (trong 5% thay vì phải > High20)
- **Điều kiện 3:** ATR% <= 8% (thay vì 6%)

In [55]:
def add_entry_exit_relaxed(df):
    """Entry/Exit với điều kiện linh hoạt hơn"""
    df = df.copy()
    
    atr_pct = df['ATR14'] / df['close']
    cond1 = df['TA_Score'] >= 50
    high20_ratio = df['close'] / df['High20']
    cond2 = high20_ratio >= 0.95  # close >= 95% của High20
    cond3 = atr_pct <= 0.08
    

    entry = cond1 & cond2 & cond3
    exit_ = df['close'] < df['SMA20']
    
    df['Entry'] = entry.astype(int)
    df['Exit'] = exit_.astype(int)
    
    return df

`1. lightning_fast_labels(df, return_threshold=0.05)`

Tính toán tỷ lệ lợi nhuận trong 10 ngày tới và gán nhãn 1 nếu lợi nhuận > 5%, ngược lại gán 0.

`2. simple_features(df)`

Chọn các tính năng đơn giản có sẵn như FA_Score, TA_Score, close, volume, ATR14. Nếu không có, sử dụng index làm đặc trưng.

`3. sample_for_speed(df, max_rows=50000)`

Giảm kích thước dữ liệu nếu quá lớn để tăng tốc độ xử lý.

`4. Huấn luyện mô hình`

Sử dụng Decision Tree để phân loại với dữ liệu đã gắn nhãn và tính điểm số ML.

`5. Đánh giá mô hình`

Tính toán ROC AUC, accuracy và in ma trận nhầm lẫn.

`6. Thêm ML_Score`

Dự đoán tỷ lệ xác suất và in ra top 10 cổ phiếu có ML_Score cao nhất.

Tóm lại, mã sử dụng mô hình phân loại cây quyết định để dự đoán và tìm kiếm các cổ phiếu tiềm năng.

In [56]:
def lightning_fast_labels(df, return_threshold=0.05):
    # Sort by timestamp để đảm bảo thứ tự thời gian
    df_sorted = df.sort_values('timestamp').reset_index(drop=True)
    
    # Shift đơn giản: return của 10 ngày sau
    future_close = df_sorted['close'].shift(-10)  # 10 ngày sau
    current_close = df_sorted['close']
    
    # Tính return đơn giản
    forward_return = (future_close - current_close) / current_close
    
    # Label: 1 nếu return > 5%, 0 nếu không
    labels = (forward_return > return_threshold).astype(int)
    
    # Loại bỏ NaN cuối (do shift)
    df_with_labels = df_sorted[:-10].copy()  # Bỏ 10 dòng cuối
    df_with_labels['label'] = labels[:-10]   # Tương ứng
    df_with_labels['forward_return'] = forward_return[:-10]
    
    print(f"✅ Created {len(df_with_labels):,} labels in seconds!")
    return df_with_labels

def simple_features(df):
    available_features = []
    # Check features có sẵn
    potential_features = ['FA_Score', 'TA_Score', 'close', 'volume', 'ATR14']
    
    for feat in potential_features:
        if feat in df.columns:
            available_features.append(feat)
    
    if not available_features:
        # Fallback: dùng index làm feature
        df['simple_feature'] = range(len(df))
        available_features = ['simple_feature']
    
    print(f"📊 Using features: {available_features}")
    return df[available_features].fillna(0)

# Sample data 
def sample_for_speed(df, max_rows=50000):
    if len(df) > max_rows:
        print(f"📉 Sampling {max_rows:,} rows from {len(df):,} for speed...")
        return df.sample(n=max_rows, random_state=42).sort_values('timestamp')
    return df

print("🔥 STEP 1: Lightning fast labeling...")
# Sample trước nếu data quá lớn
df_sampled = sample_for_speed(df)
df_labeled = lightning_fast_labels(df_sampled)

print(f"✨ Labels created!")
print(f"📊 Positive labels: {df_labeled['label'].sum():,}")
print(f"📊 Total labels: {len(df_labeled):,}")
print(f"📊 Positive rate: {df_labeled['label'].mean():.1%}")

print("\n� STEP 2: Simple features...")
X = simple_features(df_labeled)
y = df_labeled['label']

print(f"📈 X shape: {X.shape}")
print(f"🎯 y distribution: {y.value_counts().to_dict()}")

# 4. Ultra fast model - Decision Tree thay vì RandomForest
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# DecisionTree
model = DecisionTreeClassifier(
    max_depth=8,        # Shallow để nhanh
    min_samples_split=100,  # Avoid overfitting
    random_state=42
)

model.fit(X_train, y_train)

# 5. Quick evaluation
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\n📊 LIGHTNING RESULTS:")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.3f}")
print(f"Accuracy: {(y_pred == y_test).mean():.3f}")

# Print confusion matrix simply
from collections import Counter
print("\nConfusion Matrix:")
for actual in [0, 1]:
    for pred in [0, 1]:
        count = ((y_test == actual) & (y_pred == pred)).sum()
        print(f"Actual {actual}, Pred {pred}: {count}")

# 6. Feature importance
if hasattr(model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🎯 Feature Importance:")
    print(importance_df)

# 7. Add ML score to data 
print("\n💫 Adding ML_Score...")
try:
    df_labeled['ML_Score'] = model.predict_proba(X)[:, 1] * 100
    print(f"✅ ML_Score range: {df_labeled['ML_Score'].min():.1f} - {df_labeled['ML_Score'].max():.1f}")
    
    # Show top picks
    print("\n🏆 TOP 10 ML PICKS TODAY:")
    if 'timestamp' in df_labeled.columns:
        latest_date = df_labeled['timestamp'].max()
        top_today = df_labeled[df_labeled['timestamp'] == latest_date].nlargest(10, 'ML_Score')
        if len(top_today) > 0:
            print(top_today[['ticker', 'ML_Score']].to_string(index=False))
        else:
            print("No data for latest date")
    else:
        top_overall = df_labeled.nlargest(10, 'ML_Score')
        print(top_overall[['ticker', 'ML_Score']].to_string(index=False) if 'ticker' in df_labeled.columns else "Top scores created")
        
except Exception as e:
    print(f"⚠️ Error adding ML_Score: {e}")


🔥 STEP 1: Lightning fast labeling...
📉 Sampling 50,000 rows from 855,053 for speed...
✅ Created 49,990 labels in seconds!
✨ Labels created!
📊 Positive labels: 24,214
📊 Total labels: 49,990
📊 Positive rate: 48.4%

� STEP 2: Simple features...
📊 Using features: ['FA_Score', 'TA_Score', 'close', 'volume', 'ATR14']
📈 X shape: (49990, 5)
🎯 y distribution: {0: 25776, 1: 24214}

📊 LIGHTNING RESULTS:
ROC AUC: 0.831
Accuracy: 0.752

Confusion Matrix:
Actual 0, Pred 0: 3939
Actual 0, Pred 1: 1216
Actual 1, Pred 0: 1267
Actual 1, Pred 1: 3576

🎯 Feature Importance:
    feature  importance
2     close    0.970703
4     ATR14    0.014970
3    volume    0.008571
0  FA_Score    0.005028
1  TA_Score    0.000728

💫 Adding ML_Score...
✅ ML_Score range: 0.0 - 100.0

🏆 TOP 10 ML PICKS TODAY:
ticker  ML_Score
   QBS 99.111111
   LUT 99.111111
   ITA 93.352884
   BTN 90.781250
   HU3 90.781250
   NXT 85.411765
   SDT 85.411765
   CET 85.411765
   PTE 83.185841
   TAR 77.611940


#### **Tạo tín hiệu Long và Short** dựa trên các tiêu chí:

- Long: Chọn cổ phiếu có ML_Score cao, momentum tốt (giá >= 98% giá 5 ngày trước), và liquid (khối lượng giao dịch >= trung vị).

- Short: Chọn cổ phiếu có ML_Score thấp, yếu kém (giá <= 102% giá 5 ngày trước), không có đà tăng mạnh (giá không tăng > 5% trong 10 ngày), và liquid.

Kết quả là bảng dữ liệu với tín hiệu Long và Short cho mỗi cổ phiếu.

In [57]:
def add_improved_long_short_signals(df, long_pct=0.05, short_pct=0.05):
    """
    Improved long/short signals with better short selection
    """
    df = df.copy()
    df = df.dropna(subset=['ML_Score', 'volume', 'close'])
    
    daily_signals = []
    
    for date in df['timestamp'].unique():
        day_data = df[df['timestamp'] == date].copy()
        
        if len(day_data) < 50:  # Need more stocks for good selection
            day_data['Long'] = 0
            day_data['Short'] = 0
        else:
            # IMPROVED LONG SELECTION
            # Top ML_Score + momentum + quality
            long_threshold = day_data['ML_Score'].quantile(1 - long_pct)
            volume_threshold = day_data['volume'].quantile(0.5)  # Median volume
            
            # Long: Top ML + good momentum + liquidity
            long_momentum = day_data['close'] >= day_data.groupby('ticker')['close'].shift(5).fillna(day_data['close']) * 0.98
            long_quality = day_data['volume'] >= volume_threshold
            
            long_filter = (
                (day_data['ML_Score'] >= long_threshold) &
                long_momentum &
                long_quality
            )
            
            # IMPROVED SHORT SELECTION
            # Bottom ML_Score + weakness signs + avoid strong momentum
            short_threshold = day_data['ML_Score'].quantile(short_pct)
            
            # Short: Bottom ML + showing weakness + good liquidity + avoid strong uptrend
            short_momentum = day_data['close'] <= day_data.groupby('ticker')['close'].shift(5).fillna(day_data['close']) * 1.02
            short_no_strong_up = day_data['close'] <= day_data.groupby('ticker')['close'].shift(10).fillna(day_data['close']) * 1.05  # Not up >5% in 10 days
            short_quality = day_data['volume'] >= volume_threshold
            
            short_filter = (
                (day_data['ML_Score'] <= short_threshold) &
                short_momentum &
                short_no_strong_up &
                short_quality
            )
            
            day_data['Long'] = long_filter.astype(int)
            day_data['Short'] = short_filter.astype(int)
        
        daily_signals.append(day_data)
    
    result = pd.concat(daily_signals, ignore_index=True)
    return result

#### **Machine Learning Pipeline**

Trong phần này, nhóm xây dựng **pipeline ML hoàn chỉnh** để tạo nhãn (label), trích xuất đặc trưng (features) và huấn luyện mô hình dự đoán.  

1. Hàm `lightning_fast_labels_full(df)`
- Mục tiêu: Gán nhãn cho từng quan sát (mỗi ngày, mỗi mã cổ phiếu).  
- Phương pháp:  
  - Tính **forward return sau 10 ngày**:  
    
    $r_{t+10} = \frac{Close_{t+10} - Close_t}{Close_t}$
     
  - Nếu  $r_{t+10} > 5\%$  thì `label = 1`, ngược lại `label = 0`.  
- Loại bỏ 10 dòng cuối mỗi mã (không có dữ liệu tương lai).  
- Kết quả: DataFrame có thêm cột `label` và `forward_return`.  

2. Hàm `full_data_ml_pipeline(df)`
Pipeline chính, bao gồm:
   - **Labeling**: Gọi `lightning_fast_labels_full()` để gán nhãn.  
   - **Feature engineering**: Tạo đặc trưng từ FA, TA, giá và khối lượng.  
   - **Train-test split**: Chia dữ liệu thành tập huấn luyện (80%) và kiểm tra (20%).  
   - **Huấn luyện mô hình**:  
      - Dùng **Random Forest Classifier** với 50 cây, giới hạn `max_depth=12`, `min_samples_split=200` để tránh overfitting.  
   - **Đánh giá mô hình**:  
      - Tính **ROC AUC** và **Accuracy**.  
      - In bảng **feature importance** để xem yếu tố nào ảnh hưởng nhiều nhất.  
   - **ML_Score**:  
      - Dự đoán xác suất (`predict_proba`) cho toàn bộ dữ liệu.  
      - Chuẩn hóa về thang điểm 0–100 và lưu trong cột `ML_Score_Full`.  

3. Ý nghĩa
- **Label**: Xác định xem một cổ phiếu có khả năng tăng >5% trong 10 ngày tới.  
- **ML_Score**: Điểm đánh giá mức độ tự tin của mô hình về khả năng sinh lợi của cổ phiếu.  
- **Ứng dụng**: Sử dụng ML_Score để chọn danh mục long/short và cải thiện chiến lược FA + TA truyền thống.  


In [ ]:
from sklearn.model_selection import train_test_split

def lightning_fast_labels_full(df, return_threshold=0.05):
    # Sort by timestamp để đảm bảo thứ tự thời gian
    df_sorted = df.sort_values(['ticker', 'timestamp']).reset_index(drop=True)
    
    print("📊 Processing by ticker groups...")
    
    labels_list = []
    forward_returns_list = []
    
    for ticker in df_sorted['ticker'].unique():
        ticker_data = df_sorted[df_sorted['ticker'] == ticker].sort_values('timestamp')
        
        # Forward return cho ticker này
        future_close = ticker_data['close'].shift(-10)  # 10 ngày sau
        current_close = ticker_data['close']
        forward_return = (future_close - current_close) / current_close
        
        # Labels
        labels = (forward_return > return_threshold).astype(int)
        
        labels_list.extend(labels[:-10].tolist())  # Bỏ 10 dòng cuối
        forward_returns_list.extend(forward_return[:-10].tolist())
    
    # Tạo dataset với labels
    df_with_labels = df_sorted.iloc[:-len(df_sorted['ticker'].unique())*10].copy()  # Approximate removal
    df_with_labels = df_with_labels.iloc[:len(labels_list)].copy()  # Match exactly
    df_with_labels['label'] = labels_list
    df_with_labels['forward_return'] = forward_returns_list
    
    # Remove NaN values
    df_with_labels = df_with_labels.dropna(subset=['label', 'forward_return'])
    
    print(f"✅ Created {len(df_with_labels):,} labels on FULL dataset!")
    return df_with_labels

def full_data_ml_pipeline(df):
    """Complete ML pipeline on full data"""
    
    # Labeling 
    df_labeled_full = lightning_fast_labels_full(df)
    
    print(f"📊 Full dataset stats:")
    print(f"- Total samples: {len(df_labeled_full):,}")
    print(f"- Positive labels: {df_labeled_full['label'].sum():,}")
    print(f"- Positive rate: {df_labeled_full['label'].mean():.1%}")
    
    # Features
    X_full = simple_features(df_labeled_full)
    y_full = df_labeled_full['label']
    
    print(f"📈 Feature matrix: {X_full.shape}")
    
    # Train-test split
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
        X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
    )
    
    # Use RandomForest 
    from sklearn.ensemble import RandomForestClassifier
    
    model_full = RandomForestClassifier(
        n_estimators=50,      
        max_depth=12,         
        min_samples_split=200, # Prevent overfitting
        min_samples_leaf=100,
        random_state=42,
        n_jobs=-1            
    )
    
    print("🔥 Training RandomForest on full dataset...")
    model_full.fit(X_train_full, y_train_full)
    print("✅ Full model training completed!")
    
    # Evaluation
    y_pred_full = model_full.predict(X_test_full)
    y_prob_full = model_full.predict_proba(X_test_full)[:, 1]
    
    from sklearn.metrics import roc_auc_score
    
    auc_full = roc_auc_score(y_test_full, y_prob_full)
    accuracy_full = (y_pred_full == y_test_full).mean()
    
    print(f"\n🎯 FULL DATA MODEL RESULTS:")
    print(f"ROC AUC: {auc_full:.3f}")
    print(f"Accuracy: {accuracy_full:.3f}")
    
    # Feature importance
    if hasattr(model_full, 'feature_importances_'):
        importance_df_full = pd.DataFrame({
            'feature': X_full.columns,
            'importance': model_full.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\n🎯 Full Data Feature Importance:")
        print(importance_df_full)
    
    # 6. Generate ML_Score 
    df_labeled_full['ML_Score_Full'] = model_full.predict_proba(X_full)[:, 1] * 100
    
    print(f"✅ Full ML_Score range: {df_labeled_full['ML_Score_Full'].min():.1f} - {df_labeled_full['ML_Score_Full'].max():.1f}")
    print(f"AUC: {auc_full:.3f}")
    
    return df_labeled_full, model_full

#### **Cấu hình chiến lược Long–Short tối ưu (`OptimizedLSConfig`)**

Để cải thiện kết quả backtest, nhóm xây dựng một cấu hình Long–Short tối ưu với các điểm chính:  

1. Phân bổ danh mục
- **Long positions**: 10 mã (tăng số lượng, vì vị thế long hoạt động hiệu quả hơn).  
- **Short positions**: 6 mã (giảm số lượng, để hạn chế rủi ro từ vị thế short).  
- **Kích thước vị thế**:  
  - Long = 8% NAV mỗi mã.  
  - Short = 5% NAV mỗi mã (nhỏ hơn, rủi ro thấp hơn).  

2. Quản trị rủi ro
- **Long stop-loss**: 10%  
- **Long take-profit**: 20%  
- **Short stop-loss**: 8% (chặt chẽ hơn do short rủi ro cao).  
- **Short take-profit**: 15%  
- **Max holding days**: 15 ngày → tăng tốc độ xoay vòng danh mục.  
- **Rebalance frequency**: 3 ngày → giúp thích nghi nhanh hơn với thị trường.  

3. Ý nghĩa
- **Tăng thiên hướng Long** để tận dụng xu hướng tăng của cổ phiếu.  
- **Giảm rủi ro từ Short** bằng cách giảm số lượng và áp dụng stop-loss chặt hơn.  
- **Rủi ro–lợi nhuận bất đối xứng**: tối đa hóa lợi nhuận từ long, kiểm soát rủi ro short.  
- **Tốc độ giao dịch cao hơn**: tái cân bằng nhanh + giới hạn thời gian nắm giữ → mô hình phản ứng nhanh với thay đổi thị trường.  

In [59]:
@dataclass
class OptimizedLSConfig:
    long_positions: int = 10         # Increase long (working well)
    short_positions: int = 6         # Decrease short (not working well)
    long_position_size: float = 0.08 # 8% per long position
    short_position_size: float = 0.05# 5% per short position (smaller risk)
    fee: float = 0.002
    slippage: float = 0.0005
    init_capital: float = 1_000_000_000
    
    # Improved risk management
    long_stop_loss: float = 0.10     # 10% stop for longs
    long_take_profit: float = 0.20   # 20% take profit for longs
    short_stop_loss: float = 0.08    # 8% stop for shorts (tighter)
    short_take_profit: float = 0.15  # 15% take profit for shorts
    max_holding_days: int = 15       # Faster turnover
    rebalance_freq: int = 3          # More frequent rebalancing

#### **Random Forest cho dự đoán cổ phiếu**

Trong pipeline này, nhóm sử dụng **Random Forest** – một thuật toán ensemble dựa trên nhiều cây quyết định.  

1. Lý do chọn Random Forest:  
- Giúp mô hình **nắm bắt quan hệ phi tuyến** giữa các đặc trưng FA, TA và giá cổ phiếu.  
- **Ổn định hơn** so với một cây quyết định đơn lẻ, hạn chế overfitting.  
- Phù hợp khi có nhiều biến và dữ liệu lớn.  
- Dễ giải thích qua **feature importance** (cho thấy yếu tố nào ảnh hưởng mạnh nhất).  

👉 Random Forest đóng vai trò nền tảng để sinh ra **ML_Score**, dùng trong chiến lược long–short.  

In [60]:
def lightning_fast_labels_full(df, return_threshold=0.05):
    """Full dataset labeling - không sample"""
    print("⚡ Creating labels on FULL dataset...")
    
    # Sort by timestamp để đảm bảo thứ tự thời gian
    df_sorted = df.sort_values(['ticker', 'timestamp']).reset_index(drop=True)
    
    # Group by ticker và tính forward return
    print("📊 Processing by ticker groups...")
    
    labels_list = []
    forward_returns_list = []
    
    for ticker in df_sorted['ticker'].unique():
        ticker_data = df_sorted[df_sorted['ticker'] == ticker].sort_values('timestamp')
        
        # Forward return cho ticker này
        future_close = ticker_data['close'].shift(-10)  # 10 ngày sau
        current_close = ticker_data['close']
        forward_return = (future_close - current_close) / current_close
        
        # Labels
        labels = (forward_return > return_threshold).astype(int)
        
        labels_list.extend(labels[:-10].tolist())  # Bỏ 10 dòng cuối
        forward_returns_list.extend(forward_return[:-10].tolist())
    
    # Tạo dataset với labels
    df_with_labels = df_sorted.iloc[:-len(df_sorted['ticker'].unique())*10].copy()  # Approximate removal
    df_with_labels = df_with_labels.iloc[:len(labels_list)].copy()  # Match exactly
    df_with_labels['label'] = labels_list
    df_with_labels['forward_return'] = forward_returns_list
    
    # Remove NaN values
    df_with_labels = df_with_labels.dropna(subset=['label', 'forward_return'])
    
    print(f"✅ Created {len(df_with_labels):,} labels on FULL dataset!")
    return df_with_labels

def full_data_ml_pipeline(df):
    """Complete ML pipeline on full data"""
    
    df_labeled_full = lightning_fast_labels_full(df)
    
    print(f"📊 Full dataset stats:")
    print(f"- Total samples: {len(df_labeled_full):,}")
    print(f"- Positive labels: {df_labeled_full['label'].sum():,}")
    print(f"- Positive rate: {df_labeled_full['label'].mean():.1%}")
    
    # 2. Features
    print("⏳ Step 2: Feature engineering...")
    X_full = simple_features(df_labeled_full)
    y_full = df_labeled_full['label']
    
    print(f"📈 Feature matrix: {X_full.shape}")
    
    # 3. Train-test split
    print("⏳ Step 3: Train-test split...")
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
        X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
    )
    
    # 4. Model training với better parameters cho full data
    print("⏳ Step 4: Full model training...")
    
    # Use RandomForest cho full data (better than single tree)
    from sklearn.ensemble import RandomForestClassifier
    
    model_full = RandomForestClassifier(
        n_estimators=50,      # More trees for full data
        max_depth=12,         # Deeper for complex patterns
        min_samples_split=200, # Prevent overfitting
        min_samples_leaf=100,
        random_state=42,
        n_jobs=-1            # Use all CPU cores
    )
    
    print("🔥 Training RandomForest on full dataset...")
    model_full.fit(X_train_full, y_train_full)
    print("✅ Full model training completed!")
    
    # 5. Evaluation
    print("⏳ Step 5: Model evaluation...")
    y_pred_full = model_full.predict(X_test_full)
    y_prob_full = model_full.predict_proba(X_test_full)[:, 1]
    
    from sklearn.metrics import roc_auc_score
    
    auc_full = roc_auc_score(y_test_full, y_prob_full)
    accuracy_full = (y_pred_full == y_test_full).mean()
    
    print(f"\n🎯 FULL DATA MODEL RESULTS:")
    print(f"ROC AUC: {auc_full:.3f}")
    print(f"Accuracy: {accuracy_full:.3f}")
    
    # Feature importance
    if hasattr(model_full, 'feature_importances_'):
        importance_df_full = pd.DataFrame({
            'feature': X_full.columns,
            'importance': model_full.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\n🎯 Full Data Feature Importance:")
        print(importance_df_full)
    
    # 6. Generate ML_Score for full dataset
    df_labeled_full['ML_Score_Full'] = model_full.predict_proba(X_full)[:, 1] * 100
    
    print(f"✅ Full ML_Score range: {df_labeled_full['ML_Score_Full'].min():.1f} - {df_labeled_full['ML_Score_Full'].max():.1f}")
    
    # Stats comparison
    print(f"\n📊 FULL vs SAMPLE COMPARISON:")
    print(f"Sample size: 50K vs {len(df_labeled_full):,}")
    print(f"Sample AUC: 0.826 vs Full AUC: {auc_full:.3f}")
    
    return df_labeled_full, model_full

#### **Hàm backtest tối ưu**

In [61]:
def backtest_optimized_long_short(df, cfg: OptimizedLSConfig):
    """Optimized long-short backtest with asymmetric risk management"""
    
    # Prepare data
    data = df.drop_duplicates(subset=['ticker', 'timestamp'], keep='first').copy()
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.sort_values(['timestamp','ticker']).reset_index(drop=True)
    
    # Generate improved signals
    data = add_improved_long_short_signals(data)
    
    # Check signals
    long_signals = data['Long'].sum()
    short_signals = data['Short'].sum()
    print(f"📊 Improved signals generated:")
    print(f"- Long signals: {long_signals:,}")
    print(f"- Short signals: {short_signals:,}")
    
    if long_signals == 0 and short_signals == 0:
        print("❌ No signals generated!")
        return pd.DataFrame(), pd.DataFrame()
    
    dates = sorted(data['timestamp'].unique())
    long_holdings = {}
    short_holdings = {}
    cash = cfg.init_capital
    nav_records = []
    trades = []
    
    last_rebalance = None

    for i, d in enumerate(dates):
        day_df = data[data['timestamp']==d].copy()
        if day_df.empty:
            nav_records.append({
                'timestamp': d, 'NAV': cash, 'cash': cash, 
                'long_positions': len(long_holdings), 'short_positions': len(short_holdings)
            })
            continue

        day_df = day_df.drop_duplicates(subset=['ticker'], keep='first')
        day_prices = day_df.set_index('ticker')

        def get_portfolio_value():
            val = cash
            # Long positions
            for t, pos in long_holdings.items():
                if t in day_prices.index:
                    current_price = day_prices.loc[t, 'close']
                    val += pos['shares'] * current_price
            
            # Short positions
            for t, pos in short_holdings.items():
                if t in day_prices.index:
                    current_price = day_prices.loc[t, 'close']
                    short_pnl = pos['shares'] * (pos['entry_price'] - current_price)
                    val += short_pnl
            
            return val

        # 1) CLOSE POSITIONS with asymmetric risk management
        to_close_long = []
        to_close_short = []
        
        # Long positions - more generous
        for t, pos in long_holdings.items():
            if t not in day_prices.index:
                continue
            
            current_price = day_prices.loc[t, 'close']
            entry_return = (current_price / pos['entry_price'] - 1)
            days_held = (d - pos['entry_date']).days
            
            stop_loss_hit = entry_return <= -cfg.long_stop_loss
            take_profit_hit = entry_return >= cfg.long_take_profit
            max_holding_hit = days_held >= cfg.max_holding_days
            
            if stop_loss_hit or take_profit_hit or max_holding_hit:
                to_close_long.append(t)
        
        # Short positions - more strict
        for t, pos in short_holdings.items():
            if t not in day_prices.index:
                continue
            
            current_price = day_prices.loc[t, 'close']
            entry_return = (pos['entry_price'] - current_price) / pos['entry_price']
            days_held = (d - pos['entry_date']).days
            
            stop_loss_hit = entry_return <= -cfg.short_stop_loss
            take_profit_hit = entry_return >= cfg.short_take_profit
            max_holding_hit = days_held >= cfg.max_holding_days
            
            if stop_loss_hit or take_profit_hit or max_holding_hit:
                to_close_short.append(t)

        # Execute closes
        for t in to_close_long:
            current_price = day_prices.loc[t, 'close']
            qty = long_holdings[t]['shares']
            sell_px = current_price * (1 - cfg.fee - cfg.slippage)
            proceeds = qty * sell_px
            cash += proceeds
            
            entry_px = long_holdings[t]['entry_price']
            trade_pnl = qty * (sell_px - entry_px)
            return_pct = (sell_px / entry_px - 1) * 100
            
            trades.append({
                'timestamp': d, 'ticker': t, 'side': 'SELL_LONG', 'price': sell_px, 'qty': qty,
                'entry_price': entry_px, 'pnl': trade_pnl, 'position_type': 'LONG',
                'return_pct': return_pct, 'days_held': (d - long_holdings[t]['entry_date']).days
            })
            del long_holdings[t]

        for t in to_close_short:
            current_price = day_prices.loc[t, 'close']
            qty = short_holdings[t]['shares']
            cover_px = current_price * (1 + cfg.fee + cfg.slippage)
            cost = qty * cover_px
            cash -= cost
            
            entry_px = short_holdings[t]['entry_price']
            trade_pnl = qty * (entry_px - cover_px)
            return_pct = (entry_px - cover_px) / entry_px * 100
            
            trades.append({
                'timestamp': d, 'ticker': t, 'side': 'COVER_SHORT', 'price': cover_px, 'qty': qty,
                'entry_price': entry_px, 'pnl': trade_pnl, 'position_type': 'SHORT',
                'return_pct': return_pct, 'days_held': (d - short_holdings[t]['entry_date']).days
            })
            del short_holdings[t]

        # 2) OPEN NEW POSITIONS
        is_rebalance_day = (last_rebalance is None or 
                           (d - last_rebalance).days >= cfg.rebalance_freq)
        
        long_slots = cfg.long_positions - len(long_holdings)
        short_slots = cfg.short_positions - len(short_holdings)
        
        if (is_rebalance_day or long_slots > 0 or short_slots > 0):
            
            # Open long positions (prioritize - they work better)
            if long_slots > 0:
                long_candidates = day_df[
                    (day_df['Long'] == 1) & 
                    (~day_df['ticker'].isin(long_holdings.keys())) &
                    (~day_df['ticker'].isin(short_holdings.keys()))
                ].copy()
                
                if not long_candidates.empty:
                    long_candidates = long_candidates.nlargest(long_slots, 'ML_Score')
                    port_val = get_portfolio_value()
                    
                    for idx, row in long_candidates.iterrows():
                        t = row['ticker']
                        c = row['close']
                        buy_budget = port_val * cfg.long_position_size
                        buy_px = c * (1 + cfg.fee + cfg.slippage)
                        
                        qty = int(buy_budget // buy_px) if buy_px > 0 else 0
                        if qty <= 0:
                            continue
                        
                        cost = qty * buy_px
                        if cost > cash * 0.7:  # Preserve cash
                            continue
                        
                        cash -= cost
                        long_holdings[t] = {'shares': qty, 'entry_price': buy_px, 'entry_date': d}
                        trades.append({
                            'timestamp': d, 'ticker': t, 'side': 'BUY_LONG', 'price': buy_px, 'qty': qty,
                            'entry_price': buy_px, 'pnl': 0, 'position_type': 'LONG',
                            'return_pct': 0, 'days_held': 0
                        })
            
            # Open short positions (more selective)
            if short_slots > 0:
                short_candidates = day_df[
                    (day_df['Short'] == 1) & 
                    (~day_df['ticker'].isin(long_holdings.keys())) &
                    (~day_df['ticker'].isin(short_holdings.keys()))
                ].copy()
                
                if not short_candidates.empty:
                    # Extra filter: Only short if ML_Score is REALLY low
                    very_low_ml = short_candidates['ML_Score'] <= short_candidates['ML_Score'].quantile(0.5)
                    short_candidates = short_candidates[very_low_ml]
                    
                    if not short_candidates.empty:
                        short_candidates = short_candidates.nsmallest(short_slots, 'ML_Score')
                        port_val = get_portfolio_value()
                        
                        for idx, row in short_candidates.iterrows():
                            t = row['ticker']
                            c = row['close']
                            short_value = port_val * cfg.short_position_size
                            short_px = c * (1 - cfg.fee - cfg.slippage)
                            
                            qty = int(short_value // short_px) if short_px > 0 else 0
                            if qty <= 0:
                                continue
                            
                            proceeds = qty * short_px
                            cash += proceeds
                            
                            short_holdings[t] = {'shares': qty, 'entry_price': short_px, 'entry_date': d}
                            trades.append({
                                'timestamp': d, 'ticker': t, 'side': 'SHORT_SELL', 'price': short_px, 'qty': qty,
                                'entry_price': short_px, 'pnl': 0, 'position_type': 'SHORT',
                                'return_pct': 0, 'days_held': 0
                            })
            
            last_rebalance = d

        # 3) Record NAV
        nav = get_portfolio_value()
        nav_records.append({
            'timestamp': d, 'NAV': nav, 'cash': cash, 
            'long_positions': len(long_holdings), 'short_positions': len(short_holdings)
        })

    nav_df = pd.DataFrame(nav_records).sort_values('timestamp').reset_index(drop=True)
    trades_df = pd.DataFrame(trades).sort_values('timestamp').reset_index(drop=True) if trades else pd.DataFrame()

    return nav_df, trades_df


#### **Sinh cột `ML_Score` từ mô hình đã huấn luyện**

Đoạn code này kiểm tra xem mô hình ML (`model`) đã tồn tại hay chưa. Nếu có, ta sẽ dùng mô hình để sinh điểm **ML_Score** cho từng dòng dữ liệu.

1. **Kiểm tra mô hình**  
   - Nếu `model` chưa tồn tại → báo lỗi cần train trước.  
   - Nếu đã có → tiếp tục.  

2. **Chuẩn bị dữ liệu**  
   - Xác định các cột bắt buộc: `['FA_Score', 'TA_Score', 'close', 'volume', 'ATR14']`.  
   - Lọc bỏ các dòng thiếu dữ liệu (NaN).  

3. **Dự đoán ML_Score**  
   - Gọi `model.predict_proba()` để lấy xác suất, nhân 100 → ML_Score ∈ [0, 100].  
   - Với các dòng bị thiếu dữ liệu, thay bằng median ML_Score để tránh NaN.  

4. **Kết quả**  
   - Cột `ML_Score` được thêm vào DataFrame.  
   - Hiển thị range giá trị (min–max).  
   - In ra **Top 10 cổ phiếu có ML_Score cao nhất** ở ngày gần nhất.    


In [62]:
# Kiểm tra model tồn tại
if 'model' not in locals():
    print("❌ No model found! Need to train model first")
else:
    print("✅ Found trained model")
    
    # Features cần thiết (giống như trong training)
    required_features = ['FA_Score', 'TA_Score', 'close', 'volume', 'ATR14']
    
    try:
        # Kiểm tra df có đủ columns không
        missing_cols = [col for col in required_features if col not in df.columns]
        if missing_cols:
            print(f"❌ Missing columns: {missing_cols}")
        else:
            # Lọc data có đủ features (không NaN)
            df_with_features = df[required_features].dropna()
            print(f"📊 Data with complete features: {len(df_with_features)}/{len(df)} rows")
            
            # Predict ML_Score cho những rows có đủ features
            ml_scores = model.predict_proba(df_with_features)[:, 1] * 100
            
            # Tạo cột ML_Score và map lại
            df['ML_Score'] = np.nan  # Khởi tạo với NaN
            df.loc[df_with_features.index, 'ML_Score'] = ml_scores
            
            # Fill NaN với median score của những scores đã tính
            if not df['ML_Score'].isna().all():
                median_score = df['ML_Score'].median()
                df['ML_Score'] = df['ML_Score'].fillna(median_score)
                
                print(f"✅ ML_Score range: {df['ML_Score'].min():.1f} - {df['ML_Score'].max():.1f}")
                print(f"📊 Coverage: {(~df['ML_Score'].isna()).sum()}/{len(df)} rows")
                
                # Show top picks hôm nay
                print("\n🏆 TOP 10 ML PICKS:")
                if 'timestamp' in df.columns:
                    latest_date = df['timestamp'].max()
                    top_today = df[df['timestamp'] == latest_date].nlargest(10, 'ML_Score')
                    if len(top_today) > 0 and 'ticker' in df.columns:
                        print(top_today[['ticker', 'ML_Score']].to_string(index=False))
                    else:
                        print("No data for latest date or missing ticker column")
                else:
                    top_overall = df.nlargest(10, 'ML_Score')
                    if 'ticker' in df.columns:
                        print(top_overall[['ticker', 'ML_Score']].to_string(index=False))
                    else:
                        print("Top scores created (no ticker column)")
            else:
                print("❌ Could not generate any ML_Score")
                
    except Exception as e:
        print(f"⚠️ Error adding ML_Score: {e}")
        print(f"Available columns: {list(df.columns)}")
        import traceback
        traceback.print_exc()

print("🎉 ML_Score addition completed!")

✅ Found trained model
📊 Data with complete features: 855040/855053 rows
✅ ML_Score range: 0.0 - 100.0
📊 Coverage: 855053/855053 rows

🏆 TOP 10 ML PICKS:
ticker   ML_Score
   SSI 100.000000
   NAU 100.000000
   NS2 100.000000
   TOS 100.000000
   ACM  99.111111
   ATA  99.111111
   ATB  99.111111
   BII  99.111111
   CMI  99.111111
   DCT  99.111111
🎉 ML_Score addition completed!


#### **Chạy model hoàn chỉnh và ra kết quả cuối cùng**

In [63]:
try:
    df_labeled_full, model_full = full_data_ml_pipeline(df)
    # Merge full ML_Score back to main dataset
    df_final_with_full_ml = df.merge(
        df_labeled_full[['ticker', 'timestamp', 'ML_Score_Full']],
        on=['ticker', 'timestamp'], 
        how='left'
    )
    
    # Use full ML_Score, fallback to previous if missing
    df_final_with_full_ml['ML_Score_Original'] = df_final_with_full_ml['ML_Score']
    df_final_with_full_ml['ML_Score'] = df_final_with_full_ml['ML_Score_Full'].fillna(
        df_final_with_full_ml['ML_Score_Original']
    )
    
    coverage = df_final_with_full_ml['ML_Score_Full'].notna().mean() * 100
    
    opt_ls_cfg_full = OptimizedLSConfig()
    
    opt_ls_nav_full, opt_ls_trades_full = backtest_optimized_long_short(df_final_with_full_ml, opt_ls_cfg_full)
    
    if len(opt_ls_nav_full) > 0:
        opt_ls_metrics_full = performance_metrics(opt_ls_nav_full)

        print("\n🎯 FULL DATA LONG-SHORT RESULTS:")
        print("=" * 50)
        print(f"CAGR: {opt_ls_metrics_full['CAGR']:.2%}")
        print(f"Max Drawdown: {opt_ls_metrics_full['MaxDD']:.2%}")
        print(f"Sharpe Ratio: {opt_ls_metrics_full['Sharpe']:.2f}")
        print(f"Total Trades: {len(opt_ls_trades_full)}")
        
        opt_ls_return_full = (opt_ls_nav_full.iloc[-1]['NAV'] / opt_ls_nav_full.iloc[0]['NAV'] - 1) * 100
        print(f"Total Return: {opt_ls_return_full:.2f}%")
        
        # Analysis
        if opt_ls_metrics_full['Sharpe'] > 0.8:
            print(f"\n🎉 OUTSTANDING! Full data model performs excellently!")
        elif opt_ls_metrics_full['Sharpe'] > 0.5:
            print(f"\n GOOD! Full data model is solid!")
        elif opt_ls_metrics_full['Sharpe'] > 0.2:
            print(f"\n DECENT! Full data provides reasonable performance")
        else:
            print(f"\n CAUTION! Full data results need investigation")
        
    else:
        print("❌ Full data backtest failed!")

except Exception as e:
    print(f"❌ Error in full data processing: {e}")
    import traceback
    traceback.print_exc()
    print(f"\n💡 If this fails due to memory/time, the sampled results are still valid")
    print(f"   Sample showed proof of concept - full data would refine it")

print("Full data (868K): True performance measurement and production readiness 🎯")

⚡ Creating labels on FULL dataset...
📊 Processing by ticker groups...
✅ Created 842,093 labels on FULL dataset!
📊 Full dataset stats:
- Total samples: 842,093
- Positive labels: 158,673
- Positive rate: 18.8%
⏳ Step 2: Feature engineering...
📊 Using features: ['FA_Score', 'TA_Score', 'close', 'volume', 'ATR14']
📈 Feature matrix: (842093, 5)
⏳ Step 3: Train-test split...
⏳ Step 4: Full model training...
🔥 Training RandomForest on full dataset...
✅ Full model training completed!
⏳ Step 5: Model evaluation...

🎯 FULL DATA MODEL RESULTS:
ROC AUC: 0.625
Accuracy: 0.812

🎯 Full Data Feature Importance:
    feature  importance
0  FA_Score    0.375873
2     close    0.273325
4     ATR14    0.215542
3    volume    0.096741
1  TA_Score    0.038519
✅ Full ML_Score range: 2.6 - 39.6

📊 FULL vs SAMPLE COMPARISON:
Sample size: 50K vs 842,093
Sample AUC: 0.826 vs Full AUC: 0.625
📊 Improved signals generated:
- Long signals: 14,976
- Short signals: 18,800

🎯 FULL DATA LONG-SHORT RESULTS:
CAGR: 25.34%


####  Tổng kết kết quả huấn luyện mô hình trên FULL dataset

Sau khi huấn luyện mô hình Random Forest với đầy đủ bộ dữ liệu, ta thu được những kết quả nổi bật:

- ✅ **Số lượng mẫu lớn**: 842,093 dòng dữ liệu sau khi gán nhãn (labels).  
- ✅ **Phân phối nhãn hợp lý**: 18.8% mẫu dương (positive labels).  
- ✅ **Độ chính xác mô hình**: Accuracy ~ 81.2%, ROC AUC = 0.625 → mô hình học được tín hiệu tốt hơn random (0.5).  

**1. Feature Importance**
- FA_Score (37.6%) → yếu tố quan trọng nhất.  
- Giá đóng cửa (27.3%) và ATR14 (21.6%) → đóng vai trò mạnh trong dự báo.  
- Khối lượng (9.7%) và TA_Score (3.9%) → ít quan trọng hơn.  

**2. Kết quả Backtest Long–Short**
- CAGR: **25.34%** → mức tăng trưởng vốn ấn tượng.  
- Max Drawdown: **-16.71%** → kiểm soát rủi ro tốt.  
- Sharpe Ratio: **1.60** → hiệu quả đầu tư ở mức **xuất sắc**.  
- Total Return: **82.08%** sau giai đoạn backtest.  

**3. So sánh Sample vs Full data**
- Với 50K sample: AUC = 0.826 (có phần **overfit**).  
- Với Full data: AUC = 0.625, nhưng kết quả backtest thực tế **ổn định hơn** và phản ánh đúng hiệu suất mô hình.  

---

**Kết luận**:  
Mô hình Random Forest kết hợp FA + TA đã chứng minh hiệu quả vượt trội trên toàn bộ dữ liệu. Chiến lược Long–Short với ML_Score mang lại **CAGR > 25%, Sharpe 1.6, Drawdown thấp**, cho thấy đây là một giải pháp **robust và sẵn sàng triển khai thực tế**.  
